<a href="https://colab.research.google.com/github/ghanendrapiplani/ml_projects/blob/master/tflow2_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

import pandas as pd

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

print(train.head())
print(test.head())

train_y = train.pop('Species')
test_y = test.pop('Species')

print(train_y.head())
print(test_y.head())

In [51]:
feature_cols = []
for col in train:
  feature_cols.append(tf.feature_column.numeric_column(key=col))
print(feature_cols)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [0]:
def input_func(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [0]:
classifier = tf.estimator.DNNClassifier(
    feature_columns = feature_cols,
    hidden_units = [30, 10],
    n_classes = 3 # number of classes for model to choose from, in our case SPECIES = ['Setosa', 'Versicolor', 'Virginica']
)


classifier.train(
    input_fn=lambda: input_func(train, train_y, training=True),
    steps=5000)

In [0]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_func(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

In [69]:
def pred_fn(data, batch_size=256):
  return tf.data.Dataset.from_tensor_slices((dict(data))).batch(batch_size)

cols = train.keys()
predict = {}

print("Please type numeric values as prompted.")
for feature in cols:
  valid = True
  while valid: 
    val = input(feature + ": ")
    if not val.isdigit(): valid = False
  predict[feature] = [float(val)]

print(predict)
predictions = classifier.predict(input_fn=lambda: pred_fn(predict))
for pred_dict in predictions:
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]
  print(class_id)
  print(probability)
  print("The species predicted is {} with a probability of ({:.1f}%)".format(SPECIES[class_id], 100* probability))

Please type numeric values as prompted.
SepalLength: 2.1
SepalWidth: 2.3
PetalLength: 2.1
PetalWidth: 4.3
{'SepalLength': [2.1], 'SepalWidth': [2.3], 'PetalLength': [2.1], 'PetalWidth': [4.3]}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpae_h9w9q/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
2
0.77001935
The species predicted is Virginica with a probability of (77.0%)
